The database hosted by the University of Washington (on a machine named 'fatboy'; it seems very likely that I will refer to the database by this name for the rest of this notebook) is sub-divided into tables with each table corresponding to a type of astronomical object (main sequence star, white dwarf, galaxy, etc.).  While some tables represent a union of disparate objects (starsALL is the union of main sequence, RGB, white dwarf, RRLyrae, and blue horizontal branch stars), most of the objects in the database are segregated.  This is relevant, because each class of variable object has its own table.  There are separate tables for Cepheids, RR Lyrae, white dwarfs, and eclipsing binary stars.

A full list of tables on fatboy can be found here:

https://confluence.lsstcorp.org/display/SIM/Database+Schema

Currently, only the tables which are incorporated into starsALL are believed to have a realistic distribution, and even that is only over the LSST survey footprint (no guarantees are made for the northern sky).

We will use the table of RR Lyrae as an example to explain how variability works in CatSim.

As described in the example notebook

`sims_catUtils/examples/tutorials/reading_in_custom_data.ipynb`

fatboy database tables are accessed through python classes that are daughter classes of `CatalogDBObject`.  Many of these classes are already defined in 

`sims_catUtils/python/lsst/sims/catUtils/baseCatalogModels`

The class for accessing the table of RR Lyrae can be imported like so:

<b>Note:</b> These next two cells are going to generate warnings.  They are harmless.  We are working to fix them.

In [ ]:
from lsst.sims.catUtils.baseCatalogModels import RRLyStarObj

In [ ]:
rrlyDB = RRLyStarObj()

We now have a connection to the table of RR Lyrae.  This table contains information such as the position and velocity of each RR Lyra in our model Milky Way.  It also contains names associating each RR Lyra with a spectrum in `sims_sed_library`.  The mixin `PhotometryStars` allows us to integrate those spectra over the LSST bandpasses and produce magnitudes.

Below, we create a catalog of RR Lyrae and their magnitudes in the u band, outputting our catalog to the text file `baseline_catalog.txt`.

A more general and detailed description of this process can be found in the iPython notebooks in

`sims_catUtils/examples/tutorials/`

as well as in the notebook

`CatSim/CatsimTutorial_SimulationsAHM_1503.ipynb`

in the GitHub repository

https://github.com/uwssg/LSST-Tutorials

<b>Note:</b> Every time a daughter class of `InstanceCatalog` is created, it is added to a registry of such classes.  This registry does not allow duplicate entries, so running the cell below that declares `uMagBaselineClass` (or any cell that declares a daughter class of `InstanceCatalog`) will raise an exception.  If you find yourself needing to run the declaration cell below more than once, you will need to restart your kernel.

In [ ]:
import numpy
from lsst.sims.catalogs.measures.instance import InstanceCatalog
from lsst.sims.catUtils.mixins import PhotometryStars

In [ ]:
class uMagBaselineClass(InstanceCatalog, PhotometryStars):
    column_outputs = ['uniqueId', 'raJ2000', 'decJ2000', 'lsst_u']
    transformations = {'raJ2000':numpy.degrees, 'decJ2000':numpy.degrees}

In [ ]:
from lsst.sims.utils import ObservationMetaData

In [ ]:
#specify the telescope pointing
obs = ObservationMetaData(pointingRA=0.0, pointingDec=0.0,
                          boundType='circle', boundLength=2.0,
                          mjd=52350.0)

In [ ]:
baselineCat = uMagBaselineClass(rrlyDB, obs_metadata=obs)
baselineCat.write_catalog('baseline_catalog.txt')

Now we will print out our catalog to the notebook

In [ ]:
!cat baseline_catalog.txt

Variability is controlled by the mixins `VariabilityStars` and `VariabilityGalaxies`.  These mixins calculate columns `delta_lsst_u`, `delta_lsst_g`, `delta_lsst_r`, etc. and automatically add them to the baseline magnitude columns.

Below, we take the same telescope pointing, but turn on variability and calculate the magnitudes.  Compare the output below to the output from the baseline catalog above.  Note that we have output the `delta_lsst_u` column as well for comparison.

In [ ]:
from lsst.sims.catUtils.mixins import VariabilityStars

In [ ]:
class uMagVarClass(InstanceCatalog, PhotometryStars, VariabilityStars):
    column_outputs = ['uniqueId', 
                      'raJ2000', 'decJ2000', 
                      'lsst_u', 'delta_lsst_u']
    
    transformations = {'raJ2000':numpy.degrees,
                       'decJ2000':numpy.degrees}

In [ ]:
varCat = uMagVarClass(rrlyDB, obs_metadata=obs)
varCat.write_catalog('var_catalog.txt')

In [ ]:
!cat var_catalog.txt

How did `VariabilityStars` know how to calculate `delta_lsst_u`?

Each object in the `starsRRLy` table of fatboy has a column `varParamStr` which is a json representation of a dict.  This dict contains parameters governing the variability model.  One of these parameters is `varMethodName`.  This corresponds to a method defined in

`sims_catUtils/python/lsst/sims/catUtils/mixins/VariabilityMixin.py`

When `delta_lsst_u` is called for, `VariabilityStars` calls that method and passes the other parameters from `varParamStr` along as arguments.  The variability method then calculates magnitude offsets in each of the LSST bands and returns them in a dict keyed to bandpass name (i.e. `['u','g','r','i','z','y']`).

In the cell below, we query the `starsRRly` table directly to show what `varParamStr` looks like.

In [ ]:
dtype = numpy.dtype([('ra', numpy.float), ('dec', numpy.float),
                     ('varParamStr', str, 300)])
query = 'SELECT TOP 5 ra, decl, varParamStr FROM starsRRLy'
results = rrlyDB.execute_arbitrary(query, dtype=dtype)
for line in results:
    print line

In the case of RR Lyrae, the variability model is based on sets of six light curves (one for each LSST bandpass) stored in `sims_sed_library/rrly_lc/`.  These light curves contain `delta_[u,g,r,i,z,y]` as a function of mjd.  They are read in and interpolated (relative to the start time specified by `tStartMjd`) to determine the appropriate magnitude offset in each band.  `varParamStr` specifies which set of light curves corresponds to which specific RR Lyra with the `filename` parameter.

Most of our variability models are based on this interpolated light curve method.  To see how each variability model is implemented, inspect

`sims_catUtils/python/lsst/sims/catUtils/mixins/Variability.py`

We are working to develop more physically realistic spectro-temporal models, but that is still in a very early stage.